In [1]:
import os
import ee
import pickle
import geemap
import geemap.colormaps as cm
# geemap.update_package()

# ee.Authenticate()

In [2]:
try:
    # Initialize the library.
    ee.Initialize()
    print('Google Earth Engine has initialized successfully!')
except ee.EEException as e:
    print('Google Earth Engine has failed to initialize!')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

Google Earth Engine has initialized successfully!


In [3]:
# File name where the dictionary is stored
pickle_file = '../data/areas_of_interest.pickle'

# Initialize the areas_of_interest dictionary
if os.path.exists(pickle_file):
    # If the pickle file exists, load the dictionary from the file
    with open(pickle_file, 'rb') as handle:
        areas_of_interest = pickle.load(handle)
    print("Loaded areas of interest from pickle file.")
else:
    # If the pickle file does not exist, initialize an empty dictionary
    areas_of_interest = {}
    print("Initialized an empty dictionary for areas of interest.")

Loaded areas of interest from pickle file.


In [4]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [5]:
if Map.draw_last_feature is None:
    print("No new feature drawn. Nothing to retrieve.")
else:
    # Code for selecting the area
    fc = ee.FeatureCollection(Map.draw_last_feature)
    region = fc.geometry()
    coords = region.getInfo()['coordinates']
    area_name = input("Enter a name for this area: ")

    # Check if the area name already exists in the dictionary
    if area_name in areas_of_interest:
        print(f"The area name '{area_name}' already exists. Please choose a different name.")
    else:
        # Save the coordinates with area_name as ID
        areas_of_interest[area_name] = coords
        print(f"Area '{area_name}' saved successfully.")

        # Pickling the updated dictionary
        with open('areas_of_interest.pickle', 'wb') as handle:
            pickle.dump(areas_of_interest, handle, protocol=pickle.HIGHEST_PROTOCOL)
        print("Areas of interest have been pickled and saved.")


No new feature drawn. Nothing to retrieve.


In [6]:
for name, coordinates in areas_of_interest.items():
    print(f"Area: {name}, Coordinates: {coordinates}")

Area: Enschede, Coordinates: [[[6.795044, 52.175827], [6.795044, 52.267317], [6.985588, 52.267317], [6.985588, 52.175827], [6.795044, 52.175827]]]
Area: Boekelo, Coordinates: [[[6.707668, 52.184142], [6.707668, 52.232951], [6.833324, 52.232951], [6.833324, 52.184142], [6.707668, 52.184142]]]


In [7]:
DateStart = "2022-03-01"
DateEnd = "2022-03-08"
# Get user input for the area
input_area_name = input("Enter the name of the area: ")

# Check if the area name exists in the dictionary and proceed
if input_area_name in areas_of_interest:
    coordinates = areas_of_interest[input_area_name]

    myCollection = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
    .filterBounds(ee.Geometry.Polygon(coordinates)) \
    .filterDate(DateStart, DateEnd) \
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10)

    # Add your further processing of myCollection here
    print("Earth Engine query executed for the area:", input_area_name)
    listOfImages = myCollection.aggregate_array('system:index').getInfo()
    print('Number of images in the collection: ', len(listOfImages))

else:
    print(f"No coordinates found for the area '{input_area_name}'. Please enter a valid area name.")

Earth Engine query executed for the area: Boekelo
Number of images in the collection:  6


In [8]:
def calculate_centroid(coordinates):
    if not coordinates or not coordinates[0]:
        return None

    num_points = len(coordinates[0])
    sum_lat = sum([coord[1] for coord in coordinates[0]])  # Sum of all latitudes
    sum_lon = sum([coord[0] for coord in coordinates[0]])  # Sum of all longitudes

    return sum_lon / num_points, sum_lat / num_points  # Return centroid as (longitude, latitude)

centroid = calculate_centroid(coordinates)
print("Centroid (Longitude, Latitude):", centroid)


Centroid (Longitude, Latitude): (6.757930400000001, 52.2036656)


In [9]:
# img1 = myCollection.mosaic().clip(ee.Geometry.Polygon(areas_of_interest[input_area_name]))

In [10]:
# DateStart = "2023-03-01"
# DateEnd = "2023-04-01"
# # Get user input for the area
# input_area_name = input("Enter the name of the area: ")

# # Check if the area name exists in the dictionary and proceed
# if input_area_name in areas_of_interest:
#     coordinates = areas_of_interest[input_area_name]

#     # # Earth Engine query using the coordinates
#     # myCollection = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
#     #     .filterBounds(ee.Geometry.Polygon(coordinates)) \
#     #     .filterDate(DateStart, DateEnd) \
#     #     .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10)
    
    # myCollection = ee.ImageCollection("COPERNICUS/S2") \
    #     .filterBounds(ee.Geometry.Polygon(coordinates)) \
    #     .filterDate(DateStart, DateEnd) \
    #     .sort("CLOUD_COVERAGE_ASSESSMENT") \
    #     .first()

#     myCollection = ee.ImageCollection("COPERNICUS/S2_HARMONIZED") \
#     .filterBounds(ee.Geometry.Polygon(coordinates)) \
#     .filterDate(DateStart, DateEnd) \
#     .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10)

#     # Add your further processing of myCollection here
#     print("Earth Engine query executed for the area:", input_area_name)
#     # listOfImages = myCollection.aggregate_array('system:index').getInfo()
#     # print('Number of images in the collection: ', len(listOfImages))

# else:
#     print(f"No coordinates found for the area '{input_area_name}'. Please enter a valid area name.")

In [11]:
# point = ee.Geometry.Point(6.74, 52.22)

In [12]:
# dataset = ee.ImageCollection("COPERNICUS/S2").filterBounds(coordinates)

In [13]:
# image = ee.Image(dataset.filterDate("2023-03-01", "2023-04-01").sort("CLOUD_COVERAGE_ASSESSMENT").first())

# EXPLANATION REGARDING imageCOLLECTION -> IMAGE
    In Earth Engine, an ImageCollection is a collection of images, whereas an Image is a single image. Many methods in Earth Engine, like select, clip, and visualization functions, operate on single images, not collections.

To fix this error, you need to reduce your image collection to a **single image**. This is often done using reducers like **median(), mean(), or mosaic()**. The choice of reducer depends on what you're trying to achieve:

1. **median**(): Good for removing outliers, such as clouds in satellite imagery.
2. **mean**(): Calculates the average of all images, useful for general trends over time.
3. **mosaic**(): Overlays images on top of each other in the order they appear in the collection.

In [14]:
# image = ee.Image(myCollection)
magicCollectionConverter3000 = myCollection.median()
image = magicCollectionConverter3000.clip(ee.Geometry.Polygon(areas_of_interest[input_area_name]))

In [15]:
ndvi = image.expression(
    "(NIR - RED)/(RED + NIR)",
    {"NIR": image.select("B8"), 
     "RED": image.select("B4")}
    )

In [16]:
display_ndvi = {
    "min": 0,
    "max": 1,
    'palette': cm.palettes.ndvi
}

In [17]:
Map2 = geemap.Map()
Map2.add_colorbar(display_ndvi, label = 'NDVI index', layer_name="NDVI")
Map2.addLayer(ndvi, display_ndvi, 'NDVI')
Map2.set_center(centroid[0], centroid[1], 12)
Map2

Map(center=[52.2036656, 6.757930400000001], controls=(WidgetControl(options=['position', 'transparent_bg'], wi…